In [4]:
!pip install keras-tuner

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [5]:
!pip install keras-tuner
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
from tensorflow.keras.preprocessing import image
from kerastuner import RandomSearch

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [ ]:

# Load the ground truth data
train_data = pd.read_csv('/kaggle/input/isicgt/ISIC-2017_Training_Part3_GroundTruth.csv')
train_data.head()

# Load the training images
store_list = []
image_height = 350
image_width = 350
for i in tqdm(range(train_data.shape[0])):
    path = '/kaggle/input/isic2019/ISIC-2017_Training_Data/' + train_data['image_id'][i] + '.jpg'
    image_check = image.load_img(path, target_size=(image_height, image_width))
    image_check = image.img_to_array(image_check)
    # scaling the images
    image_check = image_check/255
    store_list.append(image_check)
x_train = np.array(store_list)
y_train = train_data.drop(columns=['image_id'])
y_train = y_train.to_numpy()

# Load the test data
test_data = pd.read_csv('/kaggle/input/test2017isic/ISIC-2017_Test_v2_Part3_GroundTruth.csv')
test_data.head()

# Load the test images
store_list = []
for i in tqdm(range(test_data.shape[0])):
    path = '/kaggle/input/test2017isic/ISIC-2017_Test_v2_Data/ISIC-2017_Test_v2_Data/' + test_data['image_id'][i] + '.jpg'
    image_check = image.load_img(path, target_size=(image_height, image_width))
    image_check = image.img_to_array(image_check)
    # scaling the images
    image_check = image_check/255
    store_list.append(image_check)
x_test = np.array(store_list)
y_test = test_data.drop(columns=['image_id'])
y_test = y_test.to_numpy()


 75%|███████▍  | 1495/2000 [01:04<01:31,  5.54it/s] 

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

datagen.fit(x_train)

# Define the EarlyStopping callback
es_callback = EarlyStopping(monitor='val_loss', patience=3)

In [ ]:
import keras
best_model = keras.models.load_model('/kaggle/input/best-model/isic2017.h5')

In [ ]:
# Train the best model with augmented data
history = best_model.fit(datagen.flow(x_train, y_train, batch_size=32), epochs=10, validation_data=(x_test, y_test), callbacks=[es_callback])


In [ ]:
# Evaluate the best model on the test set
score = best_model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


In [ ]:
best_model.save('/kaggle/working/isic2017cnn1.h5')

In [ ]:
os.mkdir('/kaggle/working/test')
os.mkdir('/kaggle/working/train')

In [ ]:
y_pred_prob = model.predict(x_test)
y_test = y_test.argmax(axis=1)

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob[:, 1])

In [ ]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, y_pred_prob[:, 1])

In [ ]:
for i in range(len(fpr)):
    print("Threshold: {:.2f} | FPR: {:.4f} | TPR: {:.4f}".format(thresholds[i], fpr[i], tpr[i]))

print("AUC: {:.4f}".format(auc))

In [ ]:
# Plot the ROC curve
plt.plot(fpr, tpr, label='ROC curve (area = {:.4f})'.format(auc))
plt.plot([0, 1], [0, 1], 'k--', label='Random guess')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='lower right')
plt.show()

In [ ]:
y_pred = y_pred_prob.argmax(axis=1)

from sklearn.metrics import confusion_matrix, classification_report

# Assuming y_test and y_pred are the true and predicted labels, respectively
cm = confusion_matrix(y_test, y_pred)

# Print the confusion matrix
print("Confusion matrix:")
print(cm)

# Extract metrics from the confusion matrix
tn, fp, fn, tp = cm.ravel()

# Compute precision, recall, and F1-score
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

# Print the metrics
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1-score: {:.4f}".format(f1_score))

# Generate a classification report
print("Classification report:")
print(classification_report(y_test, y_pred))

In [ ]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * precision * recall / (precision + recall)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

# Assuming y_test and y_pred are the true and predicted labels, respectively
cm = confusion_matrix(y_test, y_pred)

# Define the names of the classes
classes = ['Class 0', 'Class 1']

# Plot the confusion matrix
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.colorbar()
plt.title('Confusion Matrix')
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.xticks(np.arange(len(classes)), classes)
plt.yticks(np.arange(len(classes)), classes)

# Add values to the plot
thresh = cm.max() / 2.
for i, j in np.ndindex(cm.shape):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

# Save the image
plt.savefig('confusion_matrix.png')

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

# Assuming y_test and y_pred are the true and predicted labels, respectively
cm = confusion_matrix(y_test, y_pred)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate accuracy for each class
class_accuracy = cm.diagonal() / cm.sum(axis=1)

# Plot bar chart of accuracy for each class
fig, ax = plt.subplots()
ax.bar(np.arange(len(classes)), class_accuracy)
ax.set_xticks(np.arange(len(classes)))
ax.set_xticklabels(classes)

# Add accuracy value as text on top of each bar
for i, v in enumerate(class_accuracy):
    ax.text(i, v+0.01, f'{v:.2f}', ha='center')

# Set plot title and axis labels
ax.set_title('Accuracy for Each Class')
ax.set_xlabel('Class')
ax.set_ylabel('Accuracy')

# Show the plot
plt.show()

In [ ]:
# Make predictions on the test set
y_pred = best_model.predict(x_test)

# Print the actual and predicted data side by side
for i in range(len(y_test)):
    print('Actual:', y_test[i],'Predicted:', y_pred[i])

# Plot the training and validation loss and accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Define the model architecture with hyperparameters
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=16, max_value=64, step=16),
                     kernel_size=hp.Choice('conv_1_kernel', values=[3, 5]),
                     activation='relu',
                     input_shape=x_train[0].shape))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2, 2))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=32),
                     kernel_size=hp.Choice('conv_2_kernel', values=[3, 5]),
                     activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPool2D(2, 2))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Flatten())
    model.add(Dense(units=hp.Int('dense_1_units', min_value=64, max_value=256, step=64),
                    activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(2, activation='sigmoid'))

    model.compile(optimizer=keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    return model

# Hyperparameter tuning
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='output',
    project_name='ISIC')

tuner.search_space_summary()

# Perform the search for the best hyperparameters
tuner.search(x_train, y_train, epochs=1, validation_data=(x_test, y_test))

# Get the best model and hyperparameters
best_model = tuner.get_best_models(num_models=1)[0]
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
# Save the best model to a file
best_model.save('/kaggle/working/best_model.h5')

# Save the best hyperparameters to a JSON file
import json
with open('/kaggle/working/best_hyperparameters.json', 'w') as f:
    json.dump(best_hyperparameters.values, f)